# Lookup case

In [133]:
from pyspark.sql import SparkSession
import datetime

In [2]:
spark = SparkSession.builder.appName("Spring-Media-DEV").getOrCreate()

In [106]:
browser_lookup = spark.createDataFrame([{"id": 1, "name": "Chrome"}, {"id": 2, "name": "Safari"}, {"id": 3, "name": "Brave"}, {"id": 4, "name": "Firefox"}])
big_table = spark.createDataFrame([{"foo": 1234, "browser_id": 1}, {"foo": 3212, "browser_id": 1}, {"foo": 312, "browser_id": 2}, {"foo": 44, "browser_id": 1}])

In [107]:
browser_lookup.show()

+---+-------+
| id|   name|
+---+-------+
|  1| Chrome|
|  2| Safari|
|  3|  Brave|
|  4|Firefox|
+---+-------+



In [126]:
import random
big_table = spark.createDataFrame([{"browser_id":random.randint(1,4), "value": x} for x in range(1,10000)])

In [127]:
big_table.limit(3).show()

+----------+-----+
|browser_id|value|
+----------+-----+
|         1|    1|
|         1|    2|
|         4|    3|
+----------+-----+



### Join small - big table 

In [128]:
big_table.join(browser_lookup, browser_lookup.id == big_table.browser_id).explain()

== Physical Plan ==
*(5) SortMergeJoin [browser_id#379L], [id#364L], Inner
:- *(2) Sort [browser_id#379L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(browser_id#379L, 200)
:     +- *(1) Filter isnotnull(browser_id#379L)
:        +- Scan ExistingRDD[browser_id#379L,value#380L]
+- *(4) Sort [id#364L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(id#364L, 200)
      +- *(3) Filter isnotnull(id#364L)
         +- Scan ExistingRDD[id#364L,name#365]


#### Time

In [137]:
start = datetime.datetime.now()
big_table.join(browser_lookup, browser_lookup.id == big_table.browser_id).count()
end = datetime.datetime.now()
(end - start).microseconds

549048

### WitColumn for big - small 

In [129]:
dictionary = browser_lookup.rdd.collectAsMap()
dictionary

{1: 'Chrome', 2: 'Safari', 3: 'Brave', 4: 'Firefox'}

In [131]:
from pyspark.sql.functions import col, create_map, lit
from itertools import chain

mapping_expr = create_map([lit(x) for x in chain.from_iterable(dic.items())])

big_table.withColumn("name", mapping_expr.getItem(col("browser_id"))).explain()

== Physical Plan ==
*(1) Project [browser_id#379L, value#380L, keys: [1,2], values: [Chrome,Safari][cast(browser_id#379L as int)] AS name#416]
+- Scan ExistingRDD[browser_id#379L,value#380L]


#### Time

In [147]:
start = datetime.datetime.now()
mapping_expr = create_map([lit(x) for x in chain.from_iterable(dic.items())])
big_table.withColumn("name", mapping_expr.getItem(col("browser_id"))).count()
end = datetime.datetime.now()
(end - start).microseconds

63390